<a href="https://colab.research.google.com/github/srinivasreddyoleti/projects/blob/main/Sentiment%20Prediction%20on%20Movie%20Reviews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

sentiment_prediction_on_movie_reviews_path = kagglehub.competition_download('sentiment-prediction-on-movie-reviews')

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session



In [ ]:
train_data=pd.read_csv("/kaggle/input/sentiment-prediction-on-movie-reviews/train.csv")


In [ ]:
train_data.shape

In [ ]:
test_data=pd.read_csv("/kaggle/input/sentiment-prediction-on-movie-reviews/test.csv")

In [ ]:
test_data.shape

In [ ]:
sample_submission=pd.read_csv('/kaggle/input/sentiment-prediction-on-movie-reviews/sample.csv')

In [ ]:
train_data.head(5)

In [ ]:
test_data.head(5)

In [ ]:
train_data.info()

In [ ]:
train_data.isnull().sum()

In [ ]:
train_data['reviewText']=train_data['reviewText'].astype(str)

In [ ]:
train_data.drop(train_data.index[(train_data['reviewText']=='nan')],axis=0,inplace=True)

In [ ]:
x=train_data['reviewText']
y=train_data['sentiment']

In [ ]:
assert(x.shape[0]==y.shape[0])

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

np.random.seed(306)
plt.style.use('seaborn')

In [ ]:
def count():

   positive_review=0
   negative_review=0
   for sentiment in y:
        if sentiment=='POSITIVE':
           positive_review+=1
        else:
           negative_review+=1
   return(positive_review,negative_review)


In [ ]:
def bar_plot():

    plt.figure(figsize=(5.5,3.5))
    obj=('Positive_reviews','Negative_reviews')
    y_scale=np.arange(len(obj))
    clss=count()

    plt.bar(y_scale,clss,align='center',alpha=0.4)

    plt.xticks(y_scale,obj)
    plt.ylabel('Number of reviews')
    plt.title('Movie Reviews')
bar_plot()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import StandardScaler

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import ShuffleSplit

from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import validation_curve
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import log_loss
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import classification_report
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import roc_curve

from sklearn.pipeline import Pipeline


In [ ]:
shuffle_split_cv=ShuffleSplit(n_splits=4,test_size=0.2,random_state=42)

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.25,random_state=42)

In [ ]:
enc=LabelEncoder()

In [ ]:
tfidf=TfidfVectorizer()

In [ ]:
#tfidf=CountVectorizer()

In [ ]:
x_train_encoded=tfidf.fit_transform(x_train)

In [ ]:
x_test_encoded=tfidf.transform(x_test)

In [ ]:
y_train=enc.fit_transform(y_train)

In [ ]:
y_test=enc.fit_transform(y_test)

# DummyClassifier

In [ ]:
from sklearn.dummy import DummyClassifier

In [ ]:
dummy_clf=DummyClassifier(strategy='most_frequent',
                         random_state=42)

In [ ]:
model=dummy_clf.fit(x_train_encoded,y_train)

In [ ]:
y_pred=model.predict(x_test_encoded)

In [ ]:
print("Traning Score:",model.score(x_train_encoded,y_train))

In [ ]:
print("Testing Score:",accuracy_score(y_test,y_pred))

#  Logistic Regression Model

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
pipeline=Pipeline([('clf',LogisticRegression(max_iter=10000,random_state=42,
                                            C=np.infty))])
pipeline.fit(x_train_encoded,y_train)

In [ ]:
weights=pipeline[-1].coef_
bias=pipeline[-1].intercept_

In [ ]:
print("intercept (w_0):",weights)
print("weight vector (w1,...w_m):",bias)

In [ ]:
train_score=pipeline.score(x_train_encoded,y_train)
test_score=accuracy_score(y_test,pipeline.predict(x_test_encoded))

print("Model performance on train set:",train_score)
print("Model performance on test set:",test_score)

In [ ]:

def log_clf(solver,penalty=None):
    clf=LogisticRegression(solver=solver,penalty=penalty,
                           max_iter=1000,
                          tol=1e-5)
    log_clf_results_cv=cross_validate(clf,x_train_encoded,
                                  y_train,
                                  cv=shuffle_split_cv,
                                  scoring='precision',
                                  return_train_score=True,
                                  return_estimator=True,
                                     )

    return(pd.Series(log_clf_results_cv['test_score'],name='Logistic Regression score'))

In [ ]:
lbfgs_results_score=log_clf(solver='lbfgs',penalty='l2')

saga_results_score=log_clf(solver='saga',penalty='l2')
sag_results_score=log_clf(solver='sag',penalty='l2')
liblinear_results_score=log_clf(solver='liblinear',penalty='l2')

In [ ]:
print(f'Mean lbfgs accuracy score:{lbfgs_results_score.mean()} with std: {lbfgs_results_score.std()}')
print(f'Mean saga accuracy score:{saga_results_score.mean()} with std: {saga_results_score.std()}')
print(f'Mean sag accuracy score:{sag_results_score.mean()} with std: {sag_results_score.std()}')
print(f'Mean liblinear accuracy score:{liblinear_results_score.mean()} with std: {liblinear_results_score.std()}')

In [ ]:

c=0.0001
Clf_pipeline=Pipeline([('CLF',LogisticRegression(max_iter=1000,
                                                 solver='liblinear',penalty='l2',
                                                 C=c,tol=1e-5,random_state=42,
                                                 warm_start=True
                                                 ))])

In [ ]:
c=[0.001,0.01,0.1,1,10,100]
train_score,test_score=validation_curve(
                                        Clf_pipeline,x_train_encoded,
                                        y_train,param_name='CLF__C',
                                        param_range=c,cv=shuffle_split_cv,
                                        scoring='accuracy')

plt.plot(c,train_score.mean(axis=1),'b-x',label='Training_score')
plt.plot(c,test_score.mean(axis=1),'r-x',label='Validation_score')
plt.legend(loc='center right')
plt.title('Validation Curve with Logistic Regression')

In [ ]:
logistic_regression=LogisticRegression(penalty='l2',tol=0.01450,
                                      C=1,random_state=42,
                                      solver='saga',max_iter=100,
                                      multi_class='multinomial',
                                      warm_start=True)

In [ ]:
logistic_regression.fit(x_train_encoded,y_train)

In [ ]:
print('Training Score: ',logistic_regression.score(x_train_encoded,y_train))
print("Test Score:",logistic_regression.score(x_test_encoded,y_test))

In [ ]:
y_hat=logistic_regression.predict(x_train_encoded)


In [ ]:
y_pred=logistic_regression.predict(x_test_encoded)
cm_display=ConfusionMatrixDisplay.from_predictions(y_test,y_pred,values_format='.5g')
plt.grid(False)
plt.show()

In [ ]:
print(classification_report(y_test,y_pred))

In [ ]:
weights=logistic_regression.coef_
bias=logistic_regression.intercept_
print("Weights w:{0}".format(weights))
print("Shape of weights: {0}".format(weights.shape))
print("Bias: {0}".format(bias))

In [ ]:
plt.figure(figsize=(7,5))
plt.plot(np.arange(0,58647),weights[0,:])
plt.xlabel("Feature Index")
plt.ylabel('weight value')
plt.ylim((np.min(weights)-5),np.max(weights)+5)
plt.grid(True)

In [ ]:
num_zero_w=weights.shape[1]-np.count_nonzero(weights)
print("Number of weights with value zer0:",num_zero_w)

In [ ]:
y_scores=logistic_regression.decision_function(x_test_encoded)
precision,recalls,thresholds=precision_recall_curve(y_test,y_scores)
plt.figure(figsize=(5,4))
plt.plot(thresholds,precision[:-1],'r--',label='Precision')
plt.plot(thresholds,recalls[:-1],'g--',label='Recall')
plt.xlabel("Threshold")
plt.grid(True)

plt.legend(loc='best')
plt.show()

In [ ]:
plt.figure(figsize=(7,4))
plt.plot(recalls[:-1],precision[:-1],'b--',label='Precision')
plt.ylabel('Precision')
plt.xlabel("Recall")
plt.legend(loc='best')
plt.show()

In [ ]:
fpr,tpr,threshodlds=roc_curve(y_test,y_scores)
plt.figure(figsize=(7,4))
plt.plot(fpr,tpr,linewidth=3,label='Logistic Regression')
plt.plot([0,1],[0,1],'k--',label='baseEstimator')
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.legend()
plt.show()

# KNN Model

In [ ]:
#from sklearn.neighbors import KNeighborsClassifier

In [ ]:
#knn_classifier=KNeighborsClassifier(n_neighbors=4,n_jobs=-1)

In [ ]:
#knn_classifier.fit(x_train_encoded,y_train)

In [ ]:
#y_pred=knn_classifier.predict(x_test_encoded)

In [ ]:
#acc=accuracy_score(y_test,y_pred)
#acc

In [ ]:
'''from sklearn.model_selection import GridSearchCV
grid_params={'n_neighbors' : [9,15,30,45,60,75,90],
             'p':[2],
             'algorithm':['auto'],


            }


knn=KNeighborsClassifier()


grid_search= GridSearchCV(knn, param_grid=grid_params,
                   cv=2,
                   verbose=3,
                   return_train_score=True
                  )'''

In [ ]:
#grid_search.fit(x_train_encoded,y_train)

In [ ]:
#print(grid_search.best_params_)

In [ ]:
#grid_search.cv_results_

In [ ]:
#train_score=grid_search.cv_results_['mean_train_score']
#test_score=grid_search.cv_results_['mean_test_score']

In [ ]:
'''n=[9,15,30,45,60,75,90]
plt.plot(n,train_score,'b-x',label='Training_score')
plt.plot(n,test_score,'r-x',label='Validation_score')
plt.legend(loc='center right')
plt.title('Validation Curve with KNN Classifier')'''

In [ ]:
#model_selector=grid_search.best_estimator_
#print("Best Model: ",model_selector)

In [ ]:
#from sklearn.metrics import roc_curve,auc

#train_fpt,train_tpr,thresholds=roc_curve(y_train,grid_search_results_cv.predict_proba(x_train_encoded))
#train_fpt,train_tpr,thresholds=roc_curve(y_test,grid_search_results_cv.predict_proba(x_test_encoded))

In [ ]:
#y_pred=grid_search.predict(x_test_encoded)

In [ ]:
#acc=accuracy_score(y_test,y_pred)
#acc

In [ ]:
#import matplotlib.pyplot as plt
#from sklearn.metrics import confusion_matrix

In [ ]:
#cf_matrix=confusion_matrix(y_test,y_pred)
#cf_matrix

In [ ]:
#import seaborn as sns
#sns.heatmap(cf_matrix,annot=True)

# Support Vector Machine(SVM)

In [ ]:
#from sklearn import svm

In [ ]:
#svm_clf=svm.SVC(random_state=42,C=1,
 #              tol=0.1)

In [ ]:
#svm_clf.fit(x_train_encoded,y_train)

In [ ]:
#y_pred=svm_clf.predict(x_test_encoded)

In [ ]:
#print("Validation Score:",accuracy_score(y_test,y_pred))

In [ ]:
#print(classificationreport(y_test,y_pred))

# Decision Tree Classifier

In [ ]:
#from sklearn.tree import DecisionTreeClassifier

In [ ]:
#Dtree_clf=Pipeline([('dtc',DecisionTreeClassifier(random_state=42))])

In [ ]:
#Dtree_clf.fit(x_train_encoded,y_train)

In [ ]:
#y_pred=Dtree_clf.predict(x_test_encoded)

In [ ]:
#print("Test_score:", accuracy_score(y_test,y_pred))

In [ ]:
'''parameter={
    'dtc__criterion':['gini','entropy','log_loss']
}

dtc_grid_search=GridSearchCV(Dtree_clf,
                            cv=2,scoring='accuracy',
                            param_grid=parameter)'''

In [ ]:
#dtc_grid_search.fit(x_train_encoded,y_train)

In [ ]:
#dtc_grid_search.best_params_

In [ ]:
#dtc_grid_search.cv_results_

In [ ]:
'''parameter={
    'dtc__criterion':['entropy'],
    'dtc__splitter':['best','random'],
    'dtc__max_features':['sqrt','log2']
}

dtc_grid_search=GridSearchCV(Dtree_clf,
                            cv=2,scoring='accuracy',
                            param_grid=parameter)'''


In [ ]:
#dtc_grid_search.fit(x_train_encoded,y_train)

In [ ]:
#dtc_grid_search.cv_results_

In [ ]:
#dtc_grid_search.best_params_

In [ ]:
'''Dtree_clf=Pipeline([('dtc',DecisionTreeClassifier(random_state=42,
                                                 criterion='entropy',
                                                 max_features='sqrt',
                                                 splitter='random'))])'''

In [ ]:
'''max_depth=[100,200,300,400,500,600,700,800,900]
train_score,test_score=validation_curve(
                                        Dtree_clf,x_train_encoded,
                                        y_train,param_name='dtc__max_depth',
                                        param_range=max_depth,cv=2,
                                        scoring='accuracy')

plt.plot(max_depth,train_score.mean(axis=1),'b-x',label='Training_score')
plt.plot(max_depth,test_score.mean(axis=1),'r-x',label='Validation_score')
plt.legend(loc='center right')
plt.title('Validation Curve with Logistic Regression')'''

In [ ]:
'''Dtree_clf=Pipeline([('dtc',DecisionTreeClassifier(random_state=42,
                                                 criterion='entropy',
                                                 max_features='sqrt',
                                                 splitter='random',
                                                 max_depth=400,))])'''

In [ ]:
'''min_samples_split=np.arange(500,1200,10)
train_score,test_score=validation_curve(
                                        Dtree_clf,x_train_encoded,
                                        y_train,param_name='dtc__min_samples_split',
                                        param_range=min_samples_split,cv=2,
                                        scoring='accuracy')

plt.plot(min_samples_split,train_score.mean(axis=1),'b-x',label='Training_score')
plt.plot(min_samples_split,test_score.mean(axis=1),'r-x',label='Validation_score')
plt.legend(loc='best')
plt.title('Validation Curve with Logistic Regression')'''

In [ ]:
'''parameters={
    'dtc__min_samples_split':np.arange(500,1200,10)
}
dtc_grid_search=GridSearchCV(Dtree_clf,
                            cv=2,scoring='accuracy',
                            param_grid=parameters)'''

In [ ]:
#dtc_grid_search.fit(x_train_encoded,y_train)

In [ ]:
#print("Training Score:",dtc_grid_search.best_estimator_.score(x_train_encoded,y_train))

In [ ]:
#dtc_grid_search.best_params_

In [ ]:
#y_pred=dtc_grid_search.predict(x_test_encoded)

In [ ]:
#print("Testing Score:",accuracy_score(y_test,y_pred))

In [ ]:
#print(classification_report(y_test,y_pred))

# Bagging Classifier

In [ ]:
#from sklearn.ensemble import BaggingClassifier

In [ ]:
'''DT_clf=DecisionTreeClassifier(random_state=42,
                               criterion='entropy',
                               max_features='sqrt',
                               splitter='random',
                               max_depth=400,
                               min_samples_split=1130)'''

In [ ]:
#model=Pipeline([('bagg',BaggingClassifier(estimator=DT_clf,
 #                      warm_start=True))])

In [ ]:
#model.fit(x_train_encoded,y_train)

In [ ]:
#y_pred=model.predict(x_test_encoded)

In [ ]:
#print("Trainign Score",model.score(x_train_encoded,y_train))

In [ ]:
#print("Testing Score:", accuracy_score(y_test,y_pred))

In [ ]:
#from sklearn.metrics import classification_report

In [ ]:
#print(classification_report(y_test,y_pred))

In [ ]:
'''parameters = {
    'bagg__n_estimators': [20, 50, 100, 200, 300]
}
clf = GridSearchCV(model, parameters, cv=2,
                  scoring='accuracy',error_score=0,
                  verbose=3)
clf.fit(x_train_encoded,y_train)'''


In [ ]:
#clf.best_params_

In [ ]:
'''parameters = {
    'bagg__max_samples': [x/10 for x in range(1,11)]
}
clf = GridSearchCV(model, parameters, cv=2,
                  scoring='accuracy',error_score=0,
                  verbose=3)
clf.fit(x_train_encoded,y_train)'''

In [ ]:
#clf.best_params_

In [ ]:
'''parameters = {
    'bagg__max_features': [x/10 for x in range(1,11)]
}
clf = GridSearchCV(model, parameters, cv=2,
                  scoring='accuracy',error_score=0,
                  verbose=3)
clf.fit(x_train_encoded,y_train)'''



In [ ]:
#clf.best_params_

In [ ]:
'''parameters = {
    'bagg__n_estimators':[30,150,200,250],
    'bagg__max_samples':[0.8],
    'bagg__max_features':[0.9,0.95,1.0]
}
grid_search=GridSearchCV(model, parameters, cv=2,
                  scoring='accuracy',error_score=0,
                  verbose=3)
grid_search.fit(x_train_encoded,y_train)'''


In [ ]:
#print("Trainign Score",grid_search.score(x_train_encoded,y_train))

In [ ]:
#y_pred=grid_search.predict(x_test_encoded)

In [ ]:
#print("Testing Score:", accuracy_score(y_test,y_pred))

In [ ]:
#print(classification_report(y_test,y_pred))

# Boosting

In [ ]:
#from sklearn.ensemble import AdaBoostClassifier

#from sklearn.ensemble import GradientBoostingClassifier


In [ ]:
#ada_boost=Pipeline([('ada',AdaBoostClassifier(estimator=DT_clf,
  #  random_state=42))])

In [ ]:
#ada_boost.fit(x_train_encoded,y_train)

In [ ]:
#y_pred=ada_boost.predict(x_test_encoded)

In [ ]:
#print("Training Score:",ada_boost.score(x_train_encoded,y_train))

In [ ]:
#print("Test Score:",accuracy_score(y_test,y_pred))

In [ ]:
'''parameters={
    'ada__n_estimators':[200,300,400,500,600,700,800,900],

}
ada_grid_search=GridSearchCV(ada_boost,
                            param_grid=parameters,
                            scoring='accuracy',cv=2,
                            return_train_score=True,
                            verbose=3)'''

In [ ]:
#ada_grid_search.fit(x_train_encoded,y_train)

In [ ]:
#ada_grid_search.best_params_

In [ ]:
'''parameters={
    'ada__n_estimators':[800,900],
    'ada__learning_rate':[1e-5,1e-4,1e-3,1e-2,1]
}

ada_grid_search=GridSearchCV(ada_boost,
                            param_grid=parameters,
                            scoring='accuracy',cv=2,
                            return_train_score=True,
                            verbose=3)'''


In [ ]:
#ada_grid_search.fit(x_train_encoded,y_train)

In [ ]:
#ada_grid_search.best_params_

In [ ]:
'''parameters={
    'ada__n_estimators':[900],
    'ada__learning_rate':[0.01]
}

ada_grid_search=GridSearchCV(ada_boost,
                            param_grid=parameters,
                            scoring='precision',cv=2,
                            return_train_score=True,
                            verbose=3)'''



In [ ]:
#ada_grid_search.fit(x_train_encoded,y_train)

In [ ]:
#y_pred=ada_grid_search.best_estimator_.predict(x_test_encoded)

In [ ]:
#print("Test Score:",accuracy_score(y_test,y_pred))

In [ ]:
#print(classification_report(y_test,y_pred))

In [ ]:
#from sklearn.metrics import ConfusionMatrixDisplay

#cm_matrix=ConfusionMatrixDisplay.from_predictions(y_test,y_pred,values_format='.5g')
#plt.grid(False)

# GradientBoostingClassifier

In [ ]:
#from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
#gbc=Pipeline([('grbc',GradientBoostingClassifier(random_state=42))])

In [ ]:
#gbc.fit(x_train_encoded,y_train)

In [ ]:
#y_pred=gbc.predict(x_test_encoded)

In [ ]:
#print("Training Score:",gbc.score(x_train_encoded,y_train))

In [ ]:
#print("Test Score:",accuracy_score(y_test,y_pred))

In [ ]:
'''parameters={
    'grbc__loss':['log_loss'],
    'grbc__learning_rate':[1e-5,1e-4,1e-3,1e-2,1e-1,1]
}

gbc_grid_search=GridSearchCV(gbc,
                            param_grid=parameters,
                            scoring='precision',
                             cv=2,
                            return_train_score=True,
                            error_score=0,
                            verbose=3)'''

In [ ]:
#gbc_grid_search.fit(x_train_encoded,y_train)

In [ ]:
#gbc_grid_search.best_params_

In [ ]:
'''parameters={
    'grbc__loss':['log_loss'],
    'grbc__learning_rate':[1],
    'grbc__n_estimators':[100,300,500,700,900,1100,1300,1500]
}

gbc_grid_search=GridSearchCV(gbc,
                            param_grid=parameters,
                            scoring='precision',
                             cv=2,
                            return_train_score=True,
                            error_score=0,
                            verbose=3)'''

In [ ]:
#gbc_grid_search.fit(x_train_encoded,y_train)

In [ ]:
#gbc_grid_search.best_params_

In [ ]:
'''parameters={
    'grbc__loss':['log_loss'],
    'grbc__learning_rate':[1],
    'grbc__n_estimators':[900],
    'grbc__max_features':['sqrt','log2'],
    'grbc__warm_start':[True]

}

gbc_grid_search=GridSearchCV(gbc,
                            param_grid=parameters,
                            scoring='precision',
                             cv=2,
                            return_train_score=True,
                            error_score=0,
                            verbose=3)'''

In [ ]:
#gbc_grid_search.fit(x_train_encoded,y_train)

In [ ]:
#gbc_grid_search.best_params_

In [ ]:
#print("Training score",gbc_grid_search.best_estimator_.score(x_train_encoded,y_train))

In [ ]:
#y_pred=gbc_grid_search.best_estimator_.predict(x_test_encoded)


In [ ]:
#print("Test Score:",accuracy_score(y_test,y_pred))

In [ ]:
#print(classification_report(y_test,y_pred))

In [ ]:
#from sklearn.metrics import ConfusionMatrixDisplay

#cm_matrix=ConfusionMatrixDisplay.from_predictions(y_test,y_pred,values_format='.5g')
#plt.grid(False)

# XGBoost

In [ ]:
#from xgboost import XGBClassifier

In [ ]:
#xgb_clf=Pipeline([('xgb',XGBClassifier(random_state=42))])

In [ ]:
#xgb_clf.fit(x_train_encoded,y_train)

In [ ]:
#print("Training Score:",xgb_clf.score(x_train_encoded,y_train))

In [ ]:
#y_pred=xgb_clf.predict(x_test_encoded)

In [ ]:
#print(classification_report(y_test,y_pred))

In [ ]:
'''parameters={
    'xgb__learning_rate':[0.975],
    'xgb__gamma':[4],
    'xgb__min_child_weight':[1],
    'xgb__subsample':[0.9],
    'xgb__max_depth':[70],
    'xgb__max_delta_step':[0],
    'xgb__lambda':[1],
    'xgb__alpha':[0],
    'xgb__tree_method':['approx'],
    'xgb__objective':['binary:logitraw'],
    'xgb__max_bin':[16],
    'xgb__nthread':[1],
    'xgb__base_score':[0.5],
    'xgb__n_estimators':[100],
    'xgb__booster':['gbtree']

}


xgb_grid_search=GridSearchCV(xgb_clf,param_grid=parameters,
                            scoring='precision',error_score=0,
                             cv=2,
                            verbose=3,return_train_score=True)'''

In [ ]:
#xgb_grid_search.fit(x_train_encoded,y_train)

In [ ]:
#xgb_grid_search.best_params_

In [ ]:
#print(xgb_grid_search.best_score_)
#print(xgb_grid_search.best_estimator_)

In [ ]:
#print("Training Score:",xgb_grid_search.best_estimator_.score(x_train_encoded,y_train))

In [ ]:
#y_pred=xgb_grid_search.best_estimator_.predict(x_test_encoded)

In [ ]:
#print("Test Score",accuracy_score(y_test,y_pred))

In [ ]:
#print(classification_report(y_test,y_pred))

In [ ]:
#from sklearn.metrics import ConfusionMatrixDisplay
#ConfusionMatrixDisplay.from_predictions(y_test,y_pred)
#plt.grid(False)

# MLPClassifier

In [ ]:
#from sklearn.neural_network import MLPClassifier

In [ ]:
'''mlp_clf=Pipeline([('mlp',MLPClassifier(hidden_layer_sizes=(6,5),
                                      solver='sgd',
                                      learning_rate_init=0.01,
                                       tol=1e-3,warm_start=True,
                                       early_stopping=True,
                                       validation_fraction=0.2,
                                       max_iter=100,
                                      verbose=True))])'''

In [ ]:
#mlp_clf.fit(x_train_encoded,y_train)

In [ ]:
#y_pred=mlp_clf.predict(x_test_encoded)

In [ ]:
#print("Training Score:",mlp_clf.score(x_train_encoded,y_train))

In [ ]:
#print("Test Score",accuracy_score(y_test,y_pred))

In [ ]:
test_data.head(5)

In [ ]:
test_data['reviewText']=test_data['reviewText'].astype(str)


In [ ]:
test_data=test_data['reviewText']

In [ ]:
test_data_encoded=tfidf.transform(test_data)

In [ ]:
prediction=logistic_regression.predict(test_data_encoded)

In [ ]:
prediction

In [ ]:
prediction=np.where(prediction==1,'POSITIVE','NEGATIVE')

In [ ]:
prediction

In [ ]:
index=list(np.arange(len(test_data)))

In [ ]:
submission=pd.DataFrame({'id':index,'sentiment':prediction.flatten()})



In [ ]:
submission

In [ ]:
submission.to_csv('submission.csv',index=False)